In [1]:
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=FutureWarning)
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from NewsCorpusReader import *
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.externals import joblib
import pyLDAvis
import pyLDAvis.sklearn

### News Loaders

In [2]:
def news_list(reader):
    news_generator = reader.yieldnews()
    return [next(news_generator) for x in range(len(reader.get_df()))]
news_reader = NewsCorpusReader(newssource=['apple','bas','hk01','mingpao','oriental','passion','rthk','wwp'], doccat=['News','AnaComm'], contentcat=['Local'])
news_reader.get_files()
news = news_list(news_reader)
news_space =  [' '.join(article) for article in news]
corpus_info = news_reader.get_df()
corpus_info.Time = pd.to_datetime(corpus_info.Time)
corpus = news_space
del news, news_space

87926 Files Loaded.


In [3]:
corpus[0] # Tokenized Corpus

'稱 非建制派 whatsapp 傾偈 上任 初期 自言 改善 行政立法關係 行政長官 林鄭月娥 昨日 電台 節目 中 透露 位 非建制派 議員 常常 whatsapp 傾偈 獲得 提點 解釋 政府 工作 林鄭 表示 撫心 自問 努力 修補 關係 去見 會見 多啲 立法會 發言 發言 發言 發言 形容 做 做 很少 推辭 議員 見面 林鄭月娥 形容 努力 改善 行政立法關係 社會 發生 令 非建制派 議員 反感 事 雙方 關係 倒退 直言 議會 時常 保持 平靜 和氣 環境 容易 社會 每日 有事 發生 法庭 每日 判案 非建制派 議員 不滿意 行政立法關係 冰封 強調 難以 製造 社會 氣氛 惟有 做 實務 消除 市民 社會 負面 看法 林鄭 更 透露 非建制 陣營 鐵板 一塊 更 自爆 時常 傾偈 更 位 常常 whatsapp 溝通 聲稱 目前 政治 生態 未必 高調 特首 溝通 對話 披露 細節 指 很少 推辭 議員 見面'

In [4]:
corpus_info.head(5) # Meta Data of the Corpus

,filename,NewsID,Time,DocType,ContentType,Source
0,tokenized_stoprm/apple/News_Local_20180218_000...,00000000,2018-02-18,News,Local,apple
1,tokenized_stoprm/apple/News_Local_20180725_000...,00000001,2018-07-25,News,Local,apple
2,tokenized_stoprm/apple/News_Local_20180521_000...,00000002,2018-05-21,News,Local,apple
3,tokenized_stoprm/apple/News_Local_20180724_000...,00000003,2018-07-24,News,Local,apple
4,tokenized_stoprm/apple/News_Local_20180608_000...,00000004,2018-06-08,News,Local,apple


### Vectorizer & TF-IDF

In [5]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, min_df=20)
dtm_tfidf = tfidf_vectorizer.fit_transform(corpus)
print(dtm_tfidf.shape)

(87926, 36986)


### LDA Unsupervised Learning - Topic Model

In [6]:
# # LDA Training and Save
# n_topics = 30; doc_topic_prior = 0.53; topic_word_prior = 0.2 ## Params have been Cross Validated
# lda_tfidf = LatentDirichletAllocation(n_components=n_topics, learning_offset=20.
#                     ,doc_topic_prior= doc_topic_prior, topic_word_prior=topic_word_prior, max_iter=25, n_jobs=-1, batch_size=4096)
# topics = lda_tfidf.fit_transform(dtm_tfidf)

# lda_tfidf.score(dtm_tfidf), lda_tfidf.perplexity(dtm_tfidf) ## Print Score
# joblib.dump(lda_tfidf, 'saved/lda.pkl') ## Save LDAModel

lda_tfidf = joblib.load('saved/lda.pkl')
topics = lda_tfidf.transform(dtm_tfidf)

### LDA Visualization

In [7]:
pyLDAvis.enable_notebook()
vis_data = pyLDAvis.sklearn.prepare(lda_tfidf, dtm_tfidf, tfidf_vectorizer, sort_topics=False, mds='mmds')
pyLDAvis.display(vis_data)

### DataFrame, Showing Articles

In [8]:
topics_match = topics.argmax(axis=1)+1
topics_confidence = topics.max(axis=1)
articles = [''.join(article.split()) for article in corpus]
topic_lookup = pd.DataFrame({'topics_num':topics_match, 'confidence':topics_confidence, 'articles':articles,
                             'doctype':corpus_info.DocType, 'source':corpus_info.filename.map(lambda filename: filename.split('/')[1])}).sort_values(by='confidence', ascending=False)

n_topics = lda_tfidf.n_components
confidence_threshold_news = 1/n_topics *3 
confidence_threshold_anacomm = 1/n_topics *3
drop_index = list(topic_lookup[(topic_lookup.doctype == 'News') & (topic_lookup.confidence < confidence_threshold_news)].index)
drop_index += list(topic_lookup[(topic_lookup.doctype == 'AnaComm') & (topic_lookup.confidence < confidence_threshold_anacomm)].index)
topic_lookup = topic_lookup.drop(index=drop_index)

def lookup(topic, vocab_check=None):
    if vocab_check is None:
        display(topic_lookup[(topic_lookup.topics_num==topic)&(topic_lookup.doctype=='News')].head(10))
        display(topic_lookup[(topic_lookup.topics_num==topic)&(topic_lookup.doctype=='AnaComm')].head(10))
    else:
        display(topic_lookup[(topic_lookup.topics_num==topic)&(topic_lookup.doctype=='News')&(topic_lookup.aritcles.map(lambda article: vocab_check in article))].head(10))
        display(topic_lookup[(topic_lookup.topics_num==topic)&(topic_lookup.doctype=='AnaComm')&(topic_lookup.aritcles.map(lambda article: vocab_check in article))].head(10))        

In [27]:
lookup(2) ## Seems Topic 1 is related to Education Issues

,topics_num,confidence,articles,doctype,source
30043,2,0.353995,沙中線紅磡站鋼筋接駁疑造假港鐵無可能每個步驟政府通報蘋果日報引述消息報道港鐵沙中綫紅磡站擴建...,News,hk01
51395,2,0.334901,剪鋼筋事件路署命港鐵交報告路政署要求港鐵星期沙中線紅磡站擴建工程爆出懷疑剪鋼筋造假牆身滲水事...,News,oriental
37904,2,0.333928,沙中線紅磡站鋼筋接駁疑造假事件路政署要求港鐵周內交報告昨日日報道指港鐵沙中線紅磡站擴建地盤疑...,News,hk01
78609,2,0.332838,港鐵委獨立顧問測沙中線路署促一周內交報告馬時亨報告內容公開港鐵沙中線紅磡站擴建工程質量懷疑出...,News,wwp
2510,2,0.332066,紅磡站鋼筋造假鑿牆檢查沙中綫醜聞立法會交通事務委員會明日召開特別會議討論沙中綫紅磡站擴建工程...,News,apple
84083,2,0.331307,沙中線醜聞紅磡站查短筋擬鑿壁驗駁位港府促港鐵檢視核實須提交全面記錄港鐵沙中線工程接連出事當中...,News,wwp
82894,2,0.326698,政府踢爆三大罪港鐵高層下台責任續追究資料交警方獨立委會徹查到底記者馮健文顏晉傑港鐵沙中線紅磡...,News,wwp
25923,2,0.326429,港鐵反駁紅磡站鋼筋接駁疑造假報道內容存誤導引恐慌深表遺憾蘋果日報引述消息報道指港鐵沙中線紅磡...,News,hk01
31726,2,0.325462,沙中線議員問運房局長需否問責下台陳帆日後社會自有公論港鐵沙中線紅磡站土瓜灣站會展站近日先後揭...,News,hk01
27509,2,0.323109,沙中線港鐵五次巡查發現違規剪短鋼筋三年通報政府港鐵沙中線紅磡站月台鋼筋接駁剪短涉造假事件惟港...,News,hk01


,topics_num,confidence,articles,doctype,source
84781,2,0.299949,沙中線鑿石驗安全釋疑慮須當機立斷立法會交通事務委員會周五開會討論沙中線紅磡站擴建部分連續牆月...,AnaComm,wwp
86687,2,0.287382,加強大型基建監督政府責無旁貸港鐵公司昨日政府提交沙中線紅磡站月台鋼筋被剪短事件報告詳細交代事...,AnaComm,wwp
86151,2,0.270227,善用權力查清真相保基建工程質量港鐵沙中線紅磡站月台鋼筋被剪短懷疑有人造假事情港鐵連日解釋疑點...,AnaComm,wwp
83337,2,0.268666,徹查事件追究責任維護港鐵形象港鐵昨日召開記者會交代沙中線紅磡站工程鋼筋懷疑問題事件港鐵表明嚴...,AnaComm,wwp
26660,2,0.260128,沙中線．拆局一場黑色暴雨沖散港鐵管理層馬時亨步後塵港鐵沙中線紅磡站工程早前揭鋼筋接駁剪短涉造...,AnaComm,hk01
81587,2,0.259931,獨立調查專業檢測釋除公眾疑慮盧偉國博士香港特區立法會議員工程界香港經濟民生聯盟主席非建制派議...,AnaComm,wwp
86531,2,0.259720,徹查短筋真相挽回公眾信心梁文廣經民聯深水埗區議員特首林鄭月娥宣佈成立獨立調查委員會調查港鐵沙...,AnaComm,wwp
81644,2,0.258815,引入高科技儀器監測工程增安全信心呂明華香港中華廠商聯合會榮譽會長林倫理工業社會關注組聯席召集...,AnaComm,wwp
87508,2,0.252617,港鐵須盡快沙中線問題開誠佈公李世榮民建聯中委新社聯副理事長新界青聯智庫召集人沙田區議員港鐵沙...,AnaComm,wwp
26453,2,0.241838,港鐵地震．拆局沙中線高鐵翻版重複犯錯服務經營權模式政府今日開記招公布沙中線紅磡站涉造假爆出驚...,AnaComm,hk01


# Prediction Model, And Anacomm Recommandation

In [10]:
import jieba
jieba.load_userdict('userdict/comprehensive_userdict_mixed_v4.txt')
import re
from sklearn.metrics.pairwise import euclidean_distances, cosine_similarity
import datetime

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/wl/gwg4rnhj0ls7_zhv84fsn_hm0000gn/T/jieba.cache
Loading model cost 0.475 seconds.
Prefix dict has been built succesfully.


In [11]:
stopword_set = set()
with open('stopwords_dict/comprehensive_stopwords.txt') as file:
    for word in file.read().split('\n'):
        stopword_set.add(word)

In [28]:
news_input = '''
《蘋果日報》引述消息報道，港鐵沙中綫紅磡站擴建地盤疑有鋼筋接駁位造假，指新建月台的兩幅主要牆身約5,000個螺絲頭有移位、破損或內部被石屎填封等問題，或會無法與支撐月台層的鋼筋接駁扭緊；承建商禮頓事後安排工人將鋼筋剪短，偽裝成功接駁。'''

In [30]:
def lda_output(news_input):
    news_input = news_input.lower()
    news_input = re.sub('\d+', 'num', ' '.join(jieba.cut(news_input)))
    news_input = ' '.join([word for word in news_input.split() if word not in stopword_set])
    tfidf = tfidf_vectorizer.transform([news_input])
    proba = lda_tfidf.transform(tfidf)
    return proba

def find_anacomm(news_input, date_start=None, date_end=None, topN=1, similarity = 'euclidean_distances'):
    ## topN: number of anacomm return, similarity: 'euclidean_distances' or 'cosine_similarity'
    if (date_start is not None) & (date_end is not None):
        table = corpus_info[(corpus_info.DocType=='AnaComm')&(corpus_info.Time.map(lambda x:x<=date_end))&
                                    (corpus_info.Time.map(lambda x:x>=date_start))]
    else:
        table = corpus_info[corpus_info.DocType=='AnaComm']
    anacomm_index = table.index
    
    lda_vec = lda_output(news_input)
    
    ## Using euclidean dist or cosine similarity to compare find the most similar topic in Anacomm
    assert (similarity == 'euclidean_distances') | (similarity == 'cosine_similarity')
    if similarity == 'euclidean_distances':
        similar_model = euclidean_distances
        anacomm_index = similar_model(topics[anacomm_index,:], lda_vec).flatten().argsort()[:topN] 
    else:
        similar_model = cosine_similarity
        anacomm_index = similar_model(topics[anacomm_index,:], lda_vec).flatten().argsort()[-topN:] 

    sim = [ similar_model(topics[index].reshape(1,-1), lda_vec.reshape(1,-1)) for index in anacomm_index]
    
    return table.iloc[anacomm_index,:].index, lda_output(news_input), lda_tfidf.transform(dtm_tfidf[anacomm_index,:]), sim
    ## no. of corpus, lda_vec of news_input, anacomm output lda vec

<>:3: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
<>:3: DeprecationWarning: invalid escape sequence \d
<ipython-input-30-f540bd8e2d12>:3: DeprecationWarning: invalid escape sequence \d
  news_input = re.sub('\d+', 'num', ' '.join(jieba.cut(news_input)))


In [31]:
anacomm_info = find_anacomm(news_input, topN=5, similarity='cosine_similarity')
print('''Proba_input: {}
-----------------------------------------------------'''.format(anacomm_info[1], end='\n'))
for index, confidence, sim in zip(anacomm_info[0], anacomm_info[2], anacomm_info[3]):
    print('''
Proba_output: {}

Sim: {}: 

output_text: {}
-----------------------------------------------------
    '''.format(confidence, sim, corpus[index])
    )

Proba_input: [[0.02515427 0.24596536 0.02516983 0.02549526 0.02519091 0.02644304
  0.02476191 0.0248024  0.02628641 0.02643704 0.02584853 0.02534327
  0.03589891 0.02525866 0.0250974  0.02581651 0.0251746  0.02550546
  0.02514508 0.02546329 0.02554467 0.02635799 0.02993005 0.02515487
  0.02529709 0.02548719 0.02526882 0.02624767 0.02529496 0.02515855]]
-----------------------------------------------------

Proba_output: [0.03231087 0.05927558 0.02899482 0.02567851 0.03110802 0.02424541
 0.03386679 0.07298931 0.03028111 0.02872774 0.03579284 0.03227473
 0.02346041 0.06739849 0.05737028 0.02481721 0.03926369 0.02288805
 0.02528597 0.02444624 0.0240841  0.0233508  0.02598838 0.0252289
 0.02384174 0.02904021 0.0421435  0.02714227 0.03371409 0.02498995]

Sim: [[0.69831301]]: 

output_text: 港鐵 地震 ． 拆局 沙中線 高鐵 翻版 重複 犯錯 服務 經營權 模式 政府 今日 開 記招 公布 沙中線 紅磡站 涉 造假 爆出 驚人 調查 指出 連續牆 批准 圖則 不符 當中 螺絲頭 更少 , 更 報警 處理 路政署 署長 鍾錦華 表明 政府 港鐵 負責 沙中線 管理層 信心 政府 要求 港鐵 緊隨 政府 召開 記招 宣布 工程 總監 黃唯銘 行政總裁 梁國權 位 總經理級 人員 統統 下台 成為

In [36]:
lda_output(news_input).shape

(1, 30)